In [1]:
# 900s optimization budget (15 min)
# 21600s optimization budget (6 horas)
# balanced error accuracy (1 − balanced accuracy)

# time limit = 1/10 of the optimization budget
# memory_limit = 20GB

# used a single hold_out 33.33% for test and 66.66% training
# run autosklearn without meta-learning and without ensemble building 10 times to get the best pipelines.

In [7]:
import os
import sys
sys.path.append("../")

import pickle

from sklearn.model_selection import train_test_split
from sklearn import datasets
import pandas as pd

from autosklearn.classification import AutoSklearnClassifier
from autosklearn.metrics import accuracy, balanced_accuracy, f1_macro, f1_weighted, precision_macro, precision_weighted, recall_macro, recall_weighted 
from sklearn.metrics import confusion_matrix

from util import read_dataset, get_dataset_paths, create_result_directory


def metric_list():
    metrics=[
        accuracy,
        balanced_accuracy,
        f1_macro,
        f1_weighted,
        precision_macro,
        precision_weighted,
        recall_macro,
        recall_weighted
    ]
    return metrics

def save_experiment(result_directory, dataset_name, obj):
    df_path = result_directory + "/" + dataset_name+ "_cv_results_.csv"
    df = pd.DataFrame(obj.cv_results_)
    df.to_csv(df_path, index=False)
    
    df_path = result_directory + "/" + dataset_name+ "performance_over_time_.csv"
    df = pd.DataFrame(obj.performance_over_time_)
    df.to_csv(df_path, index=False)
    
    # pickle_path = result_directory + "/" + dataset_name+ "_automl-obj.pkl"
    # with open(pickle_path, 'wb') as f:
    #     pickle.dump(obj, f)

def ger_dataset_name(dataset_path):
    return "dataset_" + dataset_path.split("dataset_")[1].split(".pkl")[0]


def ger_directory_name(directory, dataset_name, seed):
    return directory+"/"+dataset_name+"/"+str(seed)
    
    
def ger_tmp_fold_name(dataset_name, time_left_for_this_task, seed):
    return "auto-sklearn-"+dataset_name+"_"+str(time_left_for_this_task)+"_"+str(seed)


def create_result_directory(directory_name):
    if not os.path.exists(directory_name):
        os.makedirs(directory_name)
    
    return
 
    
def if_result_directory_exit(directory_name):
    if os.path.exists(directory_name):
        print("Experiment finished")
        exit(0)
    
    return


def generate_pipelines(
    dataset_path,
    result_directory,
    time_left_for_this_task=120,
    per_run_time_limit=30,
    memory_limit=10240,
    resampling_strategy="cv",
    resampling_strategy_arguments={'fold':5},
    scoring_functions=metric_list(),
    seed=1,
    tmp_folder="auto-sklearn-tmp/"
):
    dataset_name = ger_dataset_name(dataset_path)
    directory_name = ger_directory_name(result_directory, dataset_name, seed)
    tmp_folder_name = ger_tmp_fold_name(dataset_name, time_left_for_this_task, seed)

    if_result_directory_exit(directory_name) # pass
    
    X, y, categorical_indicator, attribute_names = read_dataset(dataset_path)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
    
    automl = AutoSklearnClassifier(
        time_left_for_this_task=time_left_for_this_task,
        per_run_time_limit=per_run_time_limit,
        initial_configurations_via_metalearning=0,
        ensemble_class= None,
#         ensemble_kwargs={'ensemble_size': 1},
        memory_limit=memory_limit,
        resampling_strategy=resampling_strategy,
        resampling_strategy_arguments=resampling_strategy_arguments,
        scoring_functions=scoring_functions,
        seed=seed,
        tmp_folder=tmp_folder_name
    )
    
    print(X_train.dtypes)
    automl.fit(X_train, y_train, X_test, y_test, dataset_name=dataset_name)
    
    create_result_directory(directory_name)
    save_experiment(directory_name, dataset_name, automl)



In [3]:
dataset_directory = "../../datasets/training"
paths = get_dataset_paths(dataset_directory)
paths

['../../datasets/training/dataset_40499.pkl',
 '../../datasets/training/dataset_871.pkl',
 '../../datasets/training/dataset_40704.pkl',
 '../../datasets/training/dataset_23381.pkl',
 '../../datasets/training/dataset_735.pkl',
 '../../datasets/training/dataset_41144.pkl',
 '../../datasets/training/dataset_1538.pkl',
 '../../datasets/training/dataset_41986.pkl',
 '../../datasets/training/dataset_40994.pkl',
 '../../datasets/training/dataset_1510.pkl',
 '../../datasets/training/dataset_554.pkl',
 '../../datasets/training/dataset_37.pkl',
 '../../datasets/training/dataset_1046.pkl',
 '../../datasets/training/dataset_1116.pkl',
 '../../datasets/training/dataset_24.pkl',
 '../../datasets/training/dataset_4541.pkl',
 '../../datasets/training/dataset_1501.pkl',
 '../../datasets/training/dataset_334.pkl',
 '../../datasets/training/dataset_981.pkl',
 '../../datasets/training/dataset_40646.pkl',
 '../../datasets/training/dataset_30.pkl',
 '../../datasets/training/dataset_1481.pkl',
 '../../datase

In [4]:
generate_pipelines(paths[0], "../../results/pipeline_generation_test")

Experiment finished
V1     float64
V2     float64
V3     float64
V4     float64
V5     float64
V6     float64
V7     float64
V8     float64
V9     float64
V10    float64
V11    float64
V12    float64
V13    float64
V14    float64
V15    float64
V16    float64
V17    float64
V18    float64
V19    float64
V20    float64
V21    float64
V22    float64
V23    float64
V24    float64
V25    float64
V26    float64
V27    float64
V28    float64
V29    float64
V30    float64
V31    float64
V32    float64
V33    float64
V34    float64
V35    float64
V36    float64
V37    float64
V38    float64
V39    float64
V40    float64
dtype: object


/home/ealcobaca/Projects/dynamic_pipeline_search_space/venv/lib/python3.10/site-packages/autosklearn/data/target_validator.py:187: UserWarning: Fitting transformer with a pandas series which has the dtype category. Inverse transform may not be able preserve dtype when converting to np.ndarray
  warnings.warn(


[WARNING] [2023-02-23 00:05:05,128:Client-AutoML(1):dataset_40499] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 9, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "dataset_40499"}', seed=0, budget=0.0) RunValue(cost=1.0, time=30.014349222183228, status=<StatusType.TIMEOUT: 2>, starttime=1677121393.189805, endtime=1677121424.2333915, additional_info={'error': 'Timeout', 'configuration_origin': 'Default'})
RunKey(config_id=2, instance_id='{"task_id": "dataset_40499"}', seed=0, budget=0.0) RunValue(cost=0.15103030303030301, time=3.6221280097961426, status=<StatusType.SUCCESS: 1>, starttime=1677121425.9969003, endtime=1677121429.6350896, additional_info={'accuracy': 0.15103030303030301, 'balanced_accuracy': 0.1519238706917805, 'f1_macro': 0.1566977860413974, 'f1_weighted': 0.1557816407170486, 'precision_macro': 0.13294825074272143, 'precision_weighted': 0.13226016589625927, 'recall_macro': 0.1519238706917805, 'recall_wei

In [ ]:
def read_pkl_result(paths):
    def read(path):
        with open(path, 'rb') as f:
            x = pickle.load(f)
        aux = pd.DataFrame([x], columns=["bac"])
        aux["dataset"] = get_dataset_name(path)
        return aux
    return pd.concat([read(i) for i in paths])

In [118]:
generate_pipeline(
    dataset_path=paths[1],
    result_directory="../../results/pipeline_generation",
    time_left_for_this_task=600,
    per_run_time_limit=60,
)

[WARNING] [2022-01-23 00:59:35,594:Client-autosklearn.util.backend] Directory ../../results/pipeline_generation/dataset_871/1/autosklearn_temp/.auto-sklearn/ensembles does not exist


In [120]:
df = pd.read_csv("../../results/pipeline_generation/dataset_871/1/dataset_871pipelines.csv").sort_values(by="mean_test_score", ascending=False)
cols = [
    "rank_test_scores",
    "status",
    "mean_test_score",
    "metric_balanced_accuracy",
    "metric_f1_macro",
    "mean_fit_time",
    "budgets",
    "param_data_preprocessor:__choice__",
    "param_feature_preprocessor:__choice__",
    "param_classifier:__choice__"
]

In [139]:
df[df["status"] == "Success"][cols]

,rank_test_scores,status,mean_test_score,metric_balanced_accuracy,metric_f1_macro,mean_fit_time,budgets,param_data_preprocessor:__choice__,param_feature_preprocessor:__choice__,param_classifier:__choice__
47,1,Success,0.523610,0.522862,0.521777,1.020050,0.0,feature_type,kitchen_sinks,passive_aggressive
67,1,Success,0.523610,0.522891,0.521899,3.080477,0.0,feature_type,kitchen_sinks,qda
93,3,Success,0.522560,0.522199,0.522021,1.003785,0.0,feature_type,select_rates_classification,libsvm_svc
79,4,Success,0.520462,0.520215,0.520157,5.298594,0.0,feature_type,random_trees_embedding,adaboost
106,5,Success,0.518363,0.518145,0.518106,0.754553,0.0,feature_type,kitchen_sinks,passive_aggressive
...,...,...,...,...,...,...,...,...,...,...
46,115,Success,0.469045,0.469691,0.468154,0.852791,0.0,feature_type,feature_agglomeration,qda
44,115,Success,0.469045,0.469891,0.467398,0.909387,0.0,feature_type,random_trees_embedding,multinomial_nb
6,117,Success,0.466946,0.467907,0.464754,0.836397,0.0,feature_type,no_preprocessing,sgd
94,118,Success,0.464848,0.464319,0.463756,1.518014,0.0,feature_type,polynomial,libsvm_svc


In [140]:
df[df["status"] == "Success"][df["mean_test_score"] >= (0.523610-0.01)][cols]["param_feature_preprocessor:__choice__"].unique().shape

/home/ealcobaca/Projects/dynamic_pipeline_search_space/env3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(4,)

In [7]:
!pip3.10 install matplotlib pandas scikit-learn


  Using cached matplotlib-3.7.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.6 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached Pillow-9.4.0-cp310-cp310-manylinux_2_28_x86_64.whl (3.4 MB)
  Using cached fonttools-4.38.0-py3-none-any.whl (965 kB)
  Using cached contourpy-1.0.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (300 kB)
  Using cached kiwisolver-1.4.4-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.6 MB)

[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [12]:
import sklearn.datasets
import sklearn.metrics
import sklearn.model_selection
import autosklearn.classification


X, y = sklearn.datasets.load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = \
    sklearn.model_selection.train_test_split(X, y, random_state=1)

automl = autosklearn.classification.AutoSklearnClassifier(
    initial_configurations_via_metalearning=0,
    ensemble_class= None,
    time_left_for_this_task=120,
    per_run_time_limit=30,
    memory_limit=10240,
    resampling_strategy="cv",
    resampling_strategy_arguments={'fold':5},
    scoring_functions=metric_list(),
    seed=1,
    tmp_folder="auto-sklearn-tmp/"
)
automl.fit(X_train, y_train)
y_hat = automl.predict(X_test)

print("Accuracy score", sklearn.metrics.accuracy_score(y_test, y_hat))

[WARNING] [2023-02-23 00:19:11,244:Client-AutoML(1):940cfad3-b328-11ed-9f64-d46a6af96c1f] No valid ensemble was created. Please check the logfile for errors. Default to the best individual estimator:(1, 7, 0.0)
RunKey(config_id=1, instance_id='{"task_id": "940cfad3-b328-11ed-9f64-d46a6af96c1f"}', seed=0, budget=0.0) RunValue(cost=0.025983667409057176, time=8.847066879272461, status=<StatusType.SUCCESS: 1>, starttime=1677122239.3348706, endtime=1677122248.215216, additional_info={'accuracy': 0.025983667409057176, 'balanced_accuracy': 0.02599213785822595, 'f1_macro': 0.026047117635397608, 'f1_weighted': 0.026193466174765267, 'precision_macro': 0.024203733448648897, 'precision_weighted': 0.02450968545371407, 'recall_macro': 0.02599213785822595, 'recall_weighted': 0.025983667409057176, 'duration': 8.547394037246704, 'num_run': 2, 'train_loss': 0.0, 'configuration_origin': 'Default'})
RunKey(config_id=2, instance_id='{"task_id": "940cfad3-b328-11ed-9f64-d46a6af96c1f"}', seed=0, budget=0.0) 

Accuracy score 0.9866666666666667


In [15]:
dir(automl)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_n_features',
 '_estimator_type',
 '_get_automl_class',
 '_get_param_names',
 '_get_tags',
 '_leaderboard_columns',
 '_more_tags',
 '_n_jobs',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_resolve_ensemble_class',
 '_validate_data',
 'allow_string_features',
 'automl_',
 'build_automl',
 'classes_',
 'cv_results_',
 'dask_client',
 'dataset_compression',
 'delete_tmp_folder_after_terminate',
 'disable_evaluator_output',
 'ensemble_class',
 'ensemble_kwargs',
 'ensemble_nbest',
 'ensemble_size',
 'exclude',
 'fANOVA_input_',
 'fit',
 'fit_ensemble',
 'fit_pipeline',


In [16]:
automl.performance_over_time_

,Timestamp,single_best_optimization_score,single_best_train_score
0,2023-02-23 00:17:28,0.974016,1.0
1,2023-02-23 00:17:28,0.974016,1.0
2,2023-02-23 00:17:28,0.974016,1.0
3,2023-02-23 00:17:28,0.974016,1.0
4,2023-02-23 00:18:00,0.987379,1.0
5,2023-02-23 00:18:00,0.987379,1.0
6,2023-02-23 00:18:00,0.987379,1.0
7,2023-02-23 00:18:00,0.987379,1.0


In [17]:
pd.DataFrame(automl.cv_results_)

,mean_test_score,rank_test_scores,metric_accuracy,metric_balanced_accuracy,metric_f1_macro,metric_f1_weighted,metric_precision_macro,metric_precision_weighted,metric_recall_macro,metric_recall_weighted,...,param_data_preprocessor:feature_type:numerical_transformer:rescaling:robust_scaler:q_max,param_data_preprocessor:feature_type:numerical_transformer:rescaling:robust_scaler:q_min,param_feature_preprocessor:fast_ica:n_components,param_feature_preprocessor:kernel_pca:coef0,param_feature_preprocessor:kernel_pca:degree,param_feature_preprocessor:kernel_pca:gamma,param_feature_preprocessor:nystroem_sampler:coef0,param_feature_preprocessor:nystroem_sampler:degree,param_feature_preprocessor:nystroem_sampler:gamma,param_feature_preprocessor:select_rates_classification:mode
0,0.974016,2,0.974016,0.974008,0.973953,0.973807,0.975796,0.975490,0.974008,0.974016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.838901,6,0.838901,0.839547,0.840991,0.840851,0.856098,0.856373,0.839547,0.838901,...,0.756263,0.210852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.218263,8,0.218263,0.217391,0.155335,0.155446,0.160024,0.159760,0.217391,0.218263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,9,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.786192,7,0.786192,0.787063,0.788207,0.787505,0.798261,0.797814,0.787063,0.786192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.987379,1,0.987379,0.987478,0.987633,0.987393,0.988217,0.987842,0.987478,0.987379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.000000,9,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.947290,3,0.947290,0.947489,0.947854,0.947644,0.950877,0.950643,0.947489,0.947290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.000000,9,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.930958,4,0.930958,0.931175,0.930612,0.930512,0.932712,0.932716,0.931175,0.930958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwe


In [21]:
automl.leaderboard(detailed=True)

,rank,ensemble_weight,type,cost,duration,config_id,train_loss,seed,start_time,end_time,budget,status,data_preprocessors,feature_preprocessors,balancing_strategy,config_origin
model_id,,,,,,,,,,,,,,,,
7,1,1.0,libsvm_svc,0.012621,3.23205,6,0.0,0,1.677122e+09,1.677122e+09,0.0,StatusType.SUCCESS,[],[select_percentile_classification],none,Random Search


In [22]:
automl.show_models()

/home/ealcobaca/Projects/dynamic_pipeline_search_space/venv/lib/python3.10/site-packages/autosklearn/automl.py:2147: UserWarning: No models in the ensemble. Kindly provide an ensemble class.
  warnings.warn(


{}

In [24]:
automl.get_models_with_weights()

[(1.0,
  SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'libsvm_svc', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'select_percentile_classification', 'classifier:libsvm_svc:C': 29.6597381739565, 'classifier:libsvm_svc:gamma': 1.127803068610547, 'classifier:libsvm_svc:kernel': 'rbf', 'classifier:libsvm_svc:max_iter': -1, 'classifier:libsvm_svc:shrinking': 'False', 'classifier:libsvm_svc:tol': 0.0010532997766981521, 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'median', 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'normalize', 'feature_preprocessor:select_percentile_classification:percentile': 75.64983172189774, 'feature_preprocessor:select_percentile_classification:score_func': 'chi2'},
  dataset_properties={
    'task': 2,
    'sparse': False,
    'multilabel': False,
    'multiclass': True,
    'target_type': 'classification',
    'signed': False}

In [26]:
automl.get_models_with_weights()

[(1.0,
  SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'libsvm_svc', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'select_percentile_classification', 'classifier:libsvm_svc:C': 29.6597381739565, 'classifier:libsvm_svc:gamma': 1.127803068610547, 'classifier:libsvm_svc:kernel': 'rbf', 'classifier:libsvm_svc:max_iter': -1, 'classifier:libsvm_svc:shrinking': 'False', 'classifier:libsvm_svc:tol': 0.0010532997766981521, 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'median', 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'normalize', 'feature_preprocessor:select_percentile_classification:percentile': 75.64983172189774, 'feature_preprocessor:select_percentile_classification:score_func': 'chi2'},
  dataset_properties={
    'task': 2,
    'sparse': False,
    'multilabel': False,
    'multiclass': True,
    'target_type': 'classification',
    'signed': False}

In [34]:
automl.automl_.models_.items()

dict_items([((1, 7, 0.0), SimpleClassificationPipeline({'balancing:strategy': 'none', 'classifier:__choice__': 'libsvm_svc', 'data_preprocessor:__choice__': 'feature_type', 'feature_preprocessor:__choice__': 'select_percentile_classification', 'classifier:libsvm_svc:C': 29.6597381739565, 'classifier:libsvm_svc:gamma': 1.127803068610547, 'classifier:libsvm_svc:kernel': 'rbf', 'classifier:libsvm_svc:max_iter': -1, 'classifier:libsvm_svc:shrinking': 'False', 'classifier:libsvm_svc:tol': 0.0010532997766981521, 'data_preprocessor:feature_type:numerical_transformer:imputation:strategy': 'median', 'data_preprocessor:feature_type:numerical_transformer:rescaling:__choice__': 'normalize', 'feature_preprocessor:select_percentile_classification:percentile': 75.64983172189774, 'feature_preprocessor:select_percentile_classification:score_func': 'chi2'},
dataset_properties={
  'task': 2,
  'sparse': False,
  'multilabel': False,
  'multiclass': True,
  'target_type': 'classification',
  'signed': Fal

In [35]:
df = pd.DataFrame(automl.cv_results_)
df

,mean_test_score,rank_test_scores,metric_accuracy,metric_balanced_accuracy,metric_f1_macro,metric_f1_weighted,metric_precision_macro,metric_precision_weighted,metric_recall_macro,metric_recall_weighted,...,param_data_preprocessor:feature_type:numerical_transformer:rescaling:robust_scaler:q_max,param_data_preprocessor:feature_type:numerical_transformer:rescaling:robust_scaler:q_min,param_feature_preprocessor:fast_ica:n_components,param_feature_preprocessor:kernel_pca:coef0,param_feature_preprocessor:kernel_pca:degree,param_feature_preprocessor:kernel_pca:gamma,param_feature_preprocessor:nystroem_sampler:coef0,param_feature_preprocessor:nystroem_sampler:degree,param_feature_preprocessor:nystroem_sampler:gamma,param_feature_preprocessor:select_rates_classification:mode
0,0.974016,2,0.974016,0.974008,0.973953,0.973807,0.975796,0.975490,0.974008,0.974016,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.838901,6,0.838901,0.839547,0.840991,0.840851,0.856098,0.856373,0.839547,0.838901,...,0.756263,0.210852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.218263,8,0.218263,0.217391,0.155335,0.155446,0.160024,0.159760,0.217391,0.218263,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.000000,9,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.786192,7,0.786192,0.787063,0.788207,0.787505,0.798261,0.797814,0.787063,0.786192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.987379,1,0.987379,0.987478,0.987633,0.987393,0.988217,0.987842,0.987478,0.987379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.000000,9,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,0.947290,3,0.947290,0.947489,0.947854,0.947644,0.950877,0.950643,0.947489,0.947290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,0.000000,9,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,0.930958,4,0.930958,0.931175,0.930612,0.930512,0.932712,0.932716,0.931175,0.930958,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fwe
